# PyCaret Fraud (Low Code)

In [ ]:
!pip -q install pycaret shap imbalanced-learn plotly  # if on Colab

In [ ]:
import pandas as pd, os
from pathlib import Path
DEMO = Path('../data/raw/tiny_transactions.csv')
KAGGLE = Path('../data/raw/creditcard.csv')
DATA = KAGGLE if KAGGLE.exists() else DEMO
print('Using:', DATA)
df = pd.read_csv(DATA)
df.head()


In [ ]:
from pycaret.classification import setup, compare_models, tune_model, blend_models, evaluate_model, interpret_model, finalize_model, save_model, predict_model, pull
clf1 = setup(data=df, target='Class', session_id=42, train_size=0.8, fold=5, fold_strategy='stratifiedkfold',
             normalize=True, remove_multicollinearity=True, fix_imbalance=True, log_experiment=False, html=False)
top3 = compare_models(n_select=3, sort='AUC')
tuned = [tune_model(m, optimize='AUC') for m in top3]
blend = blend_models(estimator_list=tuned, optimize='AUC')
evaluate_model(blend)
interpret_model(blend)  # SHAP summary
final_blend = finalize_model(blend)
save_model(final_blend, '../models/pycaret_fraud_v1')
holdout_preds = predict_model(final_blend); holdout_preds.head()
